In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    Fo

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 5.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from pypdf import PdfReader

# Define the path to your PDF file
path = "/content/sample_data/Foundations_of_Machine_Learning.pdf"

#extract text from pdf
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or "" # Use or "" to handle pages with no extractable text
    return text

docs = extract_text_from_pdf(path)

In [4]:
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions
import uuid

# Initialize Chroma
client = chromadb.Client()
collection = client.get_or_create_collection("huge_doc_collection")

model = SentenceTransformer("all-MiniLM-L6-v2")

# Simple chunking
chunk_size = 200
chunks = [docs[i:i+chunk_size]
          for i in range(0, len(docs), chunk_size)]

print("Total chunks:", len(chunks))

# Create structured IDs
document_name = "ml_policy_doc"

ids = [f"{document_name}_{i}" for i in range(len(chunks))]

# Generate embeddings
embeddings = model.encode(chunks)

# Add to Chroma
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),
    ids=ids,
    metadatas=[{"source": document_name}] * len(chunks)
)

print("Total vectors stored:", collection.count())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total chunks: 3977
Total vectors stored: 3977


In [5]:
# Query Search
query = "what is machine learning"
query_embedding = model.encode([query]).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=5
)

print("\nTop Results:")
for doc in results["documents"][0]:
    print("-", doc)


Top Results:
-  to
the success of the predictions made by the learner.
Machine learning consists of designing eﬃcient and accurate prediction algo-
rithms. As in other areas of computer science, some critical measur
- mplementation or their adaptation to other learning scenarios.
Machine learning is a relatively recent ﬁeld and yet probably one of the most
active ones in computer science. Given the wide accessibili
- mple.
Since the success of a learning algorithm depends on the data used, machine
learning is inherently related to data analysis and statistics. More generally, learning
techniques are data-driven me
- te some basic deﬁnitions and to describe the use and evaluation of machine
learning algorithms in practice. Spam detection is the problem of learning to
automatically classify email messages as either
- MIT Press publication team for their help and support in
the development of this text.1 Introduction
Machine learning can be broadly deﬁned as computational methods using e